In [1]:
import numpy as np
import eduHF

# 1. Define a Molecule

A molecule is build out of different atoms. Atoms can be initiated by their respective atom type and the position in three-dimensional space (using atomic units of course)

In [2]:
H = eduHF.Atom('H', np.array([0.0, 0.0, 0.0]))
F = eduHF.Atom('F', np.array([1.74, 0.0, 0.0]))

Next, we initiated the molecule with a list of all the atoms and its charge. A negative charge leads to an anion, a positive to a cation.

In [3]:
HF = eduHF.Molecule([H, F], charge=0)

# 2. Define a Basis

We first define a dictionary containing the atom types as keys. The items are a list of tuples. Every tuple represents a Slater Function. The tuples contain first the type of function (i.e., `1s`, or `2p`) and second the exponent (often called $\zeta$ (zeta))

In [11]:
# taken from:  Robert F. Stewart, "Small Gaussian Expansions of Slater‐Type Orbitals", 
#              J. Chem. Phys. 52, 431-438 (1970) https://doi.org/10.1063/1.1672702 
base = {'H' : [('1s', 1.24)],
        'Li': [('1s', 2.69), ('2s', 0.75)],
        'Be': [('1s', 3.68), ('2s', 1.10)],
        'B' : [('1s', 4.68), ('2s', 1.45), ('2p', 1.45)],
        'C' : [('1s', 5.67), ('2s', 1.72), ('2p', 1.72)],
        'N' : [('1s', 6.67), ('2s', 1.95), ('2p', 1.95)],
        'O' : [('1s', 7.66), ('2s', 2.25), ('2p', 2.25)],
        'F' : [('1s', 8.65), ('2s', 2.55), ('2p', 2.55)],
        }

Now we build the basis set and apply it to the molecule. Wenn initiating the `Basis` we need to state the number of Gaussian functions which expand a single Slater function (parameter `ng`).

In [5]:
basisset = eduHF.BasisSet(base)
basis = eduHF.Basis(HF, basisset, ng=3)

# 3. Run the SCF

Next we initate the SCF with the molecule and the basis and prepare the integrals.

In [8]:
SCF = eduHF.SCF(HF, basis)
SCF.prepare_integrals()

When running the SCF, you can define the numver of max iterations, the convergence criteria and the type of convergence criteria.

In [10]:
SCF.run_scf(max_iter=15, convergence_crit=1e-6, convergence_type='com')

############################################################
                         SCF setup                          
############################################################
  max_iter -> 15
  convergence_crit -> 1.000000e-06
  convergence_type -> com
                      Starting SCF now                      
############################################################
iteration  energy          error           converged 
0          -102.9545867877 1.4326840454    False     
1          -103.7150808637 0.2808280996    False     
2          -103.7284628047 0.0300696026    False     
3          -103.7285967624 0.0041918466    False     
4          -103.7285992109 0.0007928026    False     
5          -103.7285993035 0.0001708333    False     
6          -103.7285993080 0.0000381872    False     
7          -103.7285993082 0.0000086260    False     
8          -103.7285993083 0.0000019551    False     
9          -103.7285993083 0.0000004436    True      
#######################

**That's it! We ran our first SCF calculation!**